In [2]:
import pandas as pd
import numpy as np

In [8]:
issues = pd.read_json("preprocessed.json")
issues.sample(5)

,repo_name,title,body,languages,tags
19,prisma,Inconsistent column data: Could not convert va...,### Bug description\n\nWe recently changed dat...,"[TypeScript, JavaScript, Shell]",inconsistent column datum convert value false ...
15,beast,basic_stream's (websocket::stream) async_conne...,### Version of Beast\r\n\r\nBoost 1.86.0\r\n\r...,"[C++, C, CMake]",basicstreams websocketstream asyncconnect comp...
6,spring-data-mongodb,Cannot add litteral array with SetUnion,"Hello,\r\n\r\nUsing SetOperators.SetUnion:\r\n...","[Java, Kotlin, Dockerfile]",add litteral array setunion hello setoperators...
1,hyper,Support forward slashes on Windows,**Is your feature request related to a problem...,"[TypeScript, Shell, NSIS]",support forward slash window feature request r...
4,vercel,[cli] Add telemetry for `vercel teams invite e...,Additional telemetry (and minimal test setup t...,"[TypeScript, JavaScript, HTML]",cli add telemetry vercel team invite email ema...


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
class Litmus:
    def __init__(self, data):
        self.data = data
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(self.data['tags'])
        
    def get_rec(self, lang, limit=5):
            language_mask = self.data['languages'].apply(
                lambda x: lang.lower() in str(x).lower()
            )
            if not any(language_mask):
                return []

            relevant_issues = self.data[language_mask]
            relevant_matrix = self.tfidf_matrix[language_mask]
            
            similarity_scores = cosine_similarity(relevant_matrix, relevant_matrix)
            
            recommendations = []
            seen_titles = set()
            
            for idx in range(len(relevant_issues)):
                similar_indices = similarity_scores[idx].argsort()[::-1][1:limit+1]
            
                for similar_idx in similar_indices:
                    issue = relevant_issues.iloc[similar_idx]
                    
                    # no dups!
                    if issue['title'] not in seen_titles:
                        recommendations.append({
                            'repo_name': issue['repo_name'],
                            'title': issue['title'],
                            'languages': issue['languages'],
                            'tags': issue['tags']
                        })
                        seen_titles.add(issue['title'])
                        
                    if len(recommendations) >= limit:
                        break
                        
                if len(recommendations) >= limit:
                    break
                
            return recommendations

In [15]:
recommender = Litmus(issues)
recommendations = recommender.get_rec('TypeScript', limit=3)
for i, rec in enumerate(recommendations, 1):
        print(f"\nRecommendation {i}:")
        print(f"Repository: {rec['repo_name']}")
        print(f"Title: {rec['title']}")
        print(f"Languages: {rec['languages']}")


Recommendation 1:
Repository: hyper
Title: Support home-relative paths by expanding `~`
Languages: ['TypeScript', 'Shell', 'NSIS']

Recommendation 2:
Repository: vercel
Title: [cli] Add telemetry for `vercel teams invite email1, email2`
Languages: ['TypeScript', 'JavaScript', 'HTML']

Recommendation 3:
Repository: hyper
Title: Support forward slashes on Windows
Languages: ['TypeScript', 'Shell', 'NSIS']
